In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML

In [2]:
t, h, g, m, w, x0, p = smp.symbols('t h g m \omega x_0 \Phi', real=True)
the, phi, psi = smp.symbols(r'\theta \phi \psi', cls=smp.Function)
the = the(t)
phi = phi(t)
psi = psi(t)
# Derivatives
the_d = smp.diff(the,t)
phi_d = smp.diff(phi,t)
psi_d = smp.diff(psi,t)
# Second derivatives
the_dd = smp.diff(the_d,t)
phi_dd = smp.diff(phi_d,t)
psi_dd = smp.diff(psi_d,t)

R3 = smp.Matrix([[smp.cos(psi),-smp.sin(psi),0],
                 [smp.sin(psi),smp.cos(psi),0],
                 [0,0,1]])

R2 = smp.Matrix([[1,0,0],
                 [0,smp.cos(the),-smp.sin(the)],
                 [0,smp.sin(the),smp.cos(the)]])

R1 = smp.Matrix([[smp.cos(phi),-smp.sin(phi),0],
                 [smp.sin(phi),smp.cos(phi),0],
                 [0,0,1]])

R = R1*R2*R3 # beta to beta_prime
R_inverse = R.T
R

Matrix([
[-sin(\phi(t))*sin(\psi(t))*cos(\theta(t)) + cos(\phi(t))*cos(\psi(t)), -sin(\phi(t))*cos(\psi(t))*cos(\theta(t)) - sin(\psi(t))*cos(\phi(t)),  sin(\phi(t))*sin(\theta(t))],
[ sin(\phi(t))*cos(\psi(t)) + sin(\psi(t))*cos(\phi(t))*cos(\theta(t)), -sin(\phi(t))*sin(\psi(t)) + cos(\phi(t))*cos(\psi(t))*cos(\theta(t)), -sin(\theta(t))*cos(\phi(t))],
[                                          sin(\psi(t))*sin(\theta(t)),                                           sin(\theta(t))*cos(\psi(t)),               cos(\theta(t))]])

In [3]:
R_inverse

Matrix([
[-sin(\phi(t))*sin(\psi(t))*cos(\theta(t)) + cos(\phi(t))*cos(\psi(t)),  sin(\phi(t))*cos(\psi(t)) + sin(\psi(t))*cos(\phi(t))*cos(\theta(t)), sin(\psi(t))*sin(\theta(t))],
[-sin(\phi(t))*cos(\psi(t))*cos(\theta(t)) - sin(\psi(t))*cos(\phi(t)), -sin(\phi(t))*sin(\psi(t)) + cos(\phi(t))*cos(\psi(t))*cos(\theta(t)), sin(\theta(t))*cos(\psi(t))],
[                                          sin(\phi(t))*sin(\theta(t)),                                          -sin(\theta(t))*cos(\phi(t)),              cos(\theta(t))]])

In [4]:
omega = smp.Matrix([phi_d*smp.sin(the)*smp.sin(psi)+the_d*smp.cos(psi),
                    phi_d*smp.sin(the)*smp.cos(psi)-the_d*smp.sin(psi),
                    phi_d*smp.cos(the)+psi_d])
omega

Matrix([
[ sin(\psi(t))*sin(\theta(t))*Derivative(\phi(t), t) + cos(\psi(t))*Derivative(\theta(t), t)],
[-sin(\psi(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\psi(t))*Derivative(\phi(t), t)],
[                             cos(\theta(t))*Derivative(\phi(t), t) + Derivative(\psi(t), t)]])

In [67]:
PG = R@smp.Matrix([0,0,1]) 
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])

In [40]:
P = -m * g * smp.Matrix([0,0,1]) 

ae = -  x0 * (w**2) * smp.Matrix([smp.cos(w*t + p),
                    0,
                    0])
Fe = - m * ae
Fe

Matrix([
[\omega**2*m*x_0*cos(\Phi + \omega*t)],
[                                   0],
[                                   0]])

In [43]:
PG.dot(ae)

-\omega**2*x_0*sin(\phi(t))*sin(\theta(t))*cos(\Phi + \omega*t)

In [48]:
r_pos = PG + smp.Matrix([x0 *smp.cos(w*t + p), 0, 0])
r_pos

Matrix([
[x_0*cos(\Phi + \omega*t) + sin(\phi(t))*sin(\theta(t))],
[                          -sin(\theta(t))*cos(\phi(t))],
[                                        cos(\theta(t))]])

In [52]:
v = smp.diff(PG, t)
v.simplify()
v

Matrix([
[sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t)],
[sin(\phi(t))*sin(\theta(t))*Derivative(\phi(t), t) - cos(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t)],
[                                                                 -sin(\theta(t))*Derivative(\theta(t), t)]])

In [59]:
v_carre = v.dot(v)
v_carre.simplify()
v_carre

(sin(\phi(t))*sin(\theta(t))*Derivative(\phi(t), t) - cos(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t))**2 + (sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t))**2 + sin(\theta(t))**2*Derivative(\theta(t), t)**2

In [66]:
r_forced = PG + smp.Matrix([x0 * smp.cos(w*t + p), 0, 0])
v_forced = smp.diff(r_forced, t)
v_forced.simplify()
v_forced

Matrix([
[-\omega*x_0*sin(\Phi + \omega*t) + sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t)],
[                                   sin(\phi(t))*sin(\theta(t))*Derivative(\phi(t), t) - cos(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t)],
[                                                                                                    -sin(\theta(t))*Derivative(\theta(t), t)]])

In [61]:
v_forced_carre = v_forced.dot(v_forced)
v_forced_carre.simplify()
v_forced_carre

(sin(\phi(t))*sin(\theta(t))*Derivative(\phi(t), t) - cos(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t))**2 + (-\omega*x_0*sin(\Phi + \omega*t) + sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t))**2 + sin(\theta(t))**2*Derivative(\theta(t), t)**2

In [62]:
v_forced_carre - v_carre

-(sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t))**2 + (-\omega*x_0*sin(\Phi + \omega*t) + sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t))**2

In [70]:
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])

In [80]:
P = - m*g * smp.Matrix([0, 0, 1])
Gamma_P_cart = PG.cross(P)
Gamma_P_cart.simplify()
Gamma_P_cart

Matrix([
[g*m*sin(\theta(t))*cos(\phi(t))],
[g*m*sin(\phi(t))*sin(\theta(t))],
[                              0]])

In [81]:
o = R_inverse @ Gamma_P_cart
o.simplify()
o

Matrix([
[ g*m*sin(\theta(t))*cos(\psi(t))],
[-g*m*sin(\psi(t))*sin(\theta(t))],
[                               0]])

In [86]:
LLL = PG.cross(v)
LLL_euler = R_inverse @ LLL
LLL_euler.simplify()
LLL_euler

Matrix([
[ sin(\psi(t))*sin(\theta(t))*Derivative(\phi(t), t) + cos(\psi(t))*Derivative(\theta(t), t)],
[-sin(\psi(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\psi(t))*Derivative(\phi(t), t)],
[                                                                                          0]])

In [87]:
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])

In [110]:
J1, J3 = smp.symbols('J1 J3', real=True)
I = smp.Matrix([[J1, 0, 0], [0, J1, 0], [0, 0, J3]])
L = I * omega
L.simplify()
L

Matrix([
[ J1*(sin(\psi(t))*sin(\theta(t))*Derivative(\phi(t), t) + cos(\psi(t))*Derivative(\theta(t), t))],
[J1*(-sin(\psi(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\psi(t))*Derivative(\phi(t), t))],
[                             J3*(cos(\theta(t))*Derivative(\phi(t), t) + Derivative(\psi(t), t))]])

In [111]:
PG.cross(P)

Matrix([
[g*m*sin(\theta(t))*cos(\phi(t))],
[g*m*sin(\phi(t))*sin(\theta(t))],
[                              0]])

In [117]:
o = omega.cross(L)
o.simplify()
o

Matrix([
[(J1 - J3)*(sin(\psi(t))*Derivative(\theta(t), t) - sin(\theta(t))*cos(\psi(t))*Derivative(\phi(t), t))*(cos(\theta(t))*Derivative(\phi(t), t) + Derivative(\psi(t), t))],
[(J1 - J3)*(cos(\theta(t))*Derivative(\phi(t), t) + Derivative(\psi(t), t))*(sin(\psi(t))*sin(\theta(t))*Derivative(\phi(t), t) + cos(\psi(t))*Derivative(\theta(t), t))],
[                                                                                                                                                                      0]])

In [7]:
Gamma_P_cart = PG.cross(P)
Gamma_P_cart

Matrix([
[g*m*sin(\theta(t))*cos(\phi(t))],
[g*m*sin(\phi(t))*sin(\theta(t))],
[                              0]])

In [8]:
Gamma_P_euler = R_inverse@Gamma_P_cart
Gamma_P_euler.simplify()
Gamma_P_euler

Matrix([
[ g*m*sin(\theta(t))*cos(\psi(t))],
[-g*m*sin(\psi(t))*sin(\theta(t))],
[                               0]])

In [9]:
Gamma_Fe_cart = PG.cross(Fe)
Gamma_Fe_cart

Matrix([
[                                                                0],
[             -\omega**2*m*x_0*cos(\Phi + \omega*t)*cos(\theta(t))],
[-\omega**2*m*x_0*sin(\theta(t))*cos(\Phi + \omega*t)*cos(\phi(t))]])

In [10]:
Gamma_Fe_cart.simplify()
Gamma_Fe_cart

Matrix([
[                                                                0],
[             -\omega**2*m*x_0*cos(\Phi + \omega*t)*cos(\theta(t))],
[-\omega**2*m*x_0*sin(\theta(t))*cos(\Phi + \omega*t)*cos(\phi(t))]])

In [11]:
Gamma_Fe_euler = R_inverse@Gamma_Fe_cart
Gamma_Fe_euler.simplify()
Gamma_Fe_euler

Matrix([
[-\omega**2*m*x_0*(sin(\phi(t))*cos(\psi(t))*cos(\theta(t)) + sin(\psi(t))*cos(\phi(t)))*cos(\Phi + \omega*t)],
[ \omega**2*m*x_0*(sin(\phi(t))*sin(\psi(t))*cos(\theta(t)) - cos(\phi(t))*cos(\psi(t)))*cos(\Phi + \omega*t)],
[                                                                                                           0]])

In [12]:
v_theta = smp.Matrix([smp.cos(psi), -smp.sin(psi), 0])
v_phi = smp.Matrix([smp.sin(the) * smp.sin(psi), smp.cos(psi) * smp.sin(the), smp.cos(the)])
v_psi = smp.Matrix([0, 0, 1])
v_phi

Matrix([
[sin(\psi(t))*sin(\theta(t))],
[sin(\theta(t))*cos(\psi(t))],
[             cos(\theta(t))]])

In [13]:
Gamma_Fe_euler.dot(v_theta).simplify()

-\omega**2*m*x_0*sin(\phi(t))*cos(\Phi + \omega*t)*cos(\theta(t))

In [14]:
Gamma_Fe_euler.dot(v_phi).simplify()

-\omega**2*m*x_0*sin(\theta(t))*cos(\Phi + \omega*t)*cos(\phi(t))

In [15]:
Gamma_Fe_cart.dot(v_psi).simplify()

-\omega**2*m*x_0*sin(\theta(t))*cos(\Phi + \omega*t)*cos(\phi(t))

In [16]:
u_z = R@ v_phi
u_z.simplify()
u_z

Matrix([
[0],
[0],
[1]])

In [17]:
Gamma_P_euler.dot(v_theta).simplify()

g*m*sin(\theta(t))

In [18]:
Gamma_P_euler.dot(v_phi).simplify()

0

In [19]:
Gamma_P_cart.dot(v_psi).simplify()

0

In [20]:
v_theta

Matrix([
[ cos(\psi(t))],
[-sin(\psi(t))],
[            0]])

In [21]:
L_O = PG.cross(smp.diff(PG, t))
L_O.simplify()
L_O

Matrix([
[-cos(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t)/4 + cos(\phi(t) + 2*\theta(t))*Derivative(\phi(t), t)/4 + cos(\phi(t))*Derivative(\theta(t), t)],
[-sin(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t)/4 + sin(\phi(t) + 2*\theta(t))*Derivative(\phi(t), t)/4 + sin(\phi(t))*Derivative(\theta(t), t)],
[                                                                                                          sin(\theta(t))**2*Derivative(\phi(t), t)]])

In [22]:
L_O = R_inverse @ L_O
L_O.simplify()
L_O

Matrix([
[ sin(\psi(t))*sin(\theta(t))*Derivative(\phi(t), t) + cos(\psi(t))*Derivative(\theta(t), t)],
[-sin(\psi(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\psi(t))*Derivative(\phi(t), t)],
[                                                                                          0]])

In [23]:
L_O.dot(v_phi).simplify()

sin(\theta(t))**2*Derivative(\phi(t), t)

In [24]:
L_O.dot(v_psi).simplify()

0

In [25]:
L_O.dot(v_theta).simplify()

Derivative(\theta(t), t)

In [26]:
Fe.dot(PG).simplify()

-\omega**2*m*x_0*sin(\phi(t))*sin(\theta(t))*cos(\Phi + \omega*t)

In [27]:
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])

In [28]:
Fe

Matrix([
[-\omega**2*m*x_0*cos(\Phi + \omega*t)],
[                                    0],
[                                    0]])

In [29]:
OP = smp.Matrix([x0 * smp.cos(w*t + p), 0, 0])

v = smp.diff(OP + PG, t)
v.simplify()
v

Matrix([
[-\omega*x_0*sin(\Phi + \omega*t) + sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t)],
[                                   sin(\phi(t))*sin(\theta(t))*Derivative(\phi(t), t) - cos(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t)],
[                                                                                                    -sin(\theta(t))*Derivative(\theta(t), t)]])

In [30]:
Lp = PG.cross(v)
Lp.simplify()
Lp

Matrix([
[                                                                                                -cos(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t)/4 + cos(\phi(t) + 2*\theta(t))*Derivative(\phi(t), t)/4 + cos(\phi(t))*Derivative(\theta(t), t)],
[-\omega*x_0*sin(\Phi + \omega*t - \theta(t))/2 - \omega*x_0*sin(\Phi + \omega*t + \theta(t))/2 - sin(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t)/4 + sin(\phi(t) + 2*\theta(t))*Derivative(\phi(t), t)/4 + sin(\phi(t))*Derivative(\theta(t), t)],
[                                                                                                                                            (-\omega*x_0*sin(\Phi + \omega*t)*cos(\phi(t)) + sin(\theta(t))*Derivative(\phi(t), t))*sin(\theta(t))]])

In [31]:
Lp.dot(Fe).simplify()

\omega**2*m*x_0*(cos(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t) - cos(\phi(t) + 2*\theta(t))*Derivative(\phi(t), t) - 4*cos(\phi(t))*Derivative(\theta(t), t))*cos(\Phi + \omega*t)/4

In [32]:
poids = smp.Matrix([ 0, 0, -m*g])

PG.dot(poids).simplify()

-g*m*cos(\theta(t))

In [33]:
a = PG.cross(poids)
a

Matrix([
[g*m*sin(\theta(t))*cos(\phi(t))],
[g*m*sin(\phi(t))*sin(\theta(t))],
[                              0]])

In [34]:
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])

In [35]:
v

Matrix([
[-\omega*x_0*sin(\Phi + \omega*t) + sin(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t) + sin(\theta(t))*cos(\phi(t))*Derivative(\phi(t), t)],
[                                   sin(\phi(t))*sin(\theta(t))*Derivative(\phi(t), t) - cos(\phi(t))*cos(\theta(t))*Derivative(\theta(t), t)],
[                                                                                                    -sin(\theta(t))*Derivative(\theta(t), t)]])

In [36]:
www = smp.diff(Lp, t)
www.simplify()
www

Matrix([
[                                                                                                                                                                         (Derivative(\phi(t), t) - 2*Derivative(\theta(t), t))*sin(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t)/4 - (Derivative(\phi(t), t) + 2*Derivative(\theta(t), t))*sin(\phi(t) + 2*\theta(t))*Derivative(\phi(t), t)/4 - sin(\phi(t))*Derivative(\phi(t), t)*Derivative(\theta(t), t) - cos(\phi(t) - 2*\theta(t))*Derivative(\phi(t), (t, 2))/4 + cos(\phi(t) + 2*\theta(t))*Derivative(\phi(t), (t, 2))/4 + cos(\phi(t))*Derivative(\theta(t), (t, 2))],
[-\omega*x_0*(\omega - Derivative(\theta(t), t))*cos(\Phi + \omega*t - \theta(t))/2 - \omega*x_0*(\omega + Derivative(\theta(t), t))*cos(\Phi + \omega*t + \theta(t))/2 - (Derivative(\phi(t), t) - 2*Derivative(\theta(t), t))*cos(\phi(t) - 2*\theta(t))*Derivative(\phi(t), t)/4 + (Derivative(\phi(t), t) + 2*Derivative(\theta(t), t))*cos(\phi(t) + 2*\theta(t))*Derivative(\phi(t), 

In [38]:
PG.dot(Fe)

-\omega**2*m*x_0*sin(\phi(t))*sin(\theta(t))*cos(\Phi + \omega*t)

In [39]:
PG

Matrix([
[ sin(\phi(t))*sin(\theta(t))],
[-sin(\theta(t))*cos(\phi(t))],
[              cos(\theta(t))]])